### Import packages

In [4]:
pip install -e ..

Obtaining file:///datadrive/code/ecmwf-forecast
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for stactools-ecmwf-forecast (pyproject.toml) ... done
  Created wheel for stactools-ecmwf-forecast: filename=stactools_ecmwf_forecast-0.1.0-0.editable-py3-none-any.whl size=2701 sha256=b1056658ce5bcebd9832f20a2e81f273d8a3d03f01fb5f2c89db13a073a540f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-znyqk6wg/wheels/3c/b5/ed/198e6195eec3f29011301fb54db9bc02a9d68b2f3012f669fd
Successfully built stactools-ecmwf-forecast
  Attempting uninstall: stactools-ecmwf-forecast
    Found existing installation: stactools-ecmwf-forecast 0.1.0
    Uninstalling stactools-ecmwf-forecast-0.1.0:
      Successfully uninstalled stactools-ecmwf-forecast-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
from azure.storage.blob import BlobServiceClient
import xarray as xr
import os
import fsspec
from stactools.ecmwf_forecast import stac

/anaconda/envs/py38_default/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


### Set up storage account connection string

In [2]:
connection_string =  getpass.getpass('<storage-account-connection-string>')
container = 'ecmwf'
account_name = 'orbstore'
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_container_client = blob_service_client.get_container_client(container)

<storage-account-connection-string> ········


### Create STAC item

In [3]:
filepath = 'https://orbstore.blob.core.windows.net/ecmwf/20220121/00z/0p4-beta/wave/20220121000000-0h-wave-fc.grib2'
item = stac.create_item([filepath], connection_string = connection_string)

/anaconda/envs/py38_default/lib/python3.8/site-packages/kerchunk/combine.py:269: UserWarning: Concatenated coordinate 'valid_time' contains less than expectednumber of values across the datasets: [1642723200]
  warnings.warn(


### Load dataset via kerchunk indices

In [4]:
%%time
m = fsspec.get_mapper('reference://', fo=item.properties['kerchunk_indices'], remote_options={"connection_string": connection_string})
ds = xr.open_dataset(m, engine='zarr')
ds['swh'].mean()

CPU times: user 53.5 ms, sys: 8.39 ms, total: 61.9 ms
Wall time: 83 ms


<timed exec>:2: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.


<xarray.DataArray 'swh' ()>
array(2.72163539)

### Load dataset via original .GRIB2 file

In [5]:
%%time

import urllib.request
import xarray as xr

url = 'https://ai4edataeuwest.blob.core.windows.net/ecmwf/20220121/00z/0p4-beta/wave/20220121000000-0h-wave-fc.grib2'
filename, _ = urllib.request.urlretrieve(url)

ds = xr.open_dataset(filename, engine="cfgrib")
ds['swh'].mean()

CPU times: user 785 ms, sys: 59.7 ms, total: 845 ms
Wall time: 2.36 s


<xarray.DataArray 'swh' ()>
array(2.7216356, dtype=float32)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    meanSea     float64 ...
    valid_time  datetime64[ns] ...